In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import subprocess, cv2, json, os, sys, shutil, pyk4a, time
import numpy as np, matplotlib.pyplot as plt
from kinectacq.acquisition import start_recording
from kinectacq.paths import DATA_DIR, ensure_dir

### Set up recording location

In [3]:
import datetime

In [4]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
timestamp

'2022-02-18_13-45-10'

In [5]:
filename_prefix = DATA_DIR / 'test_recording' / timestamp

In [6]:
ensure_dir(filename_prefix)

### Get camera information

In [7]:
!k4arecorder --list

Index:0	Serial:000774310512	Color:1.6.102	Depth:1.6.75
Index:1	Serial:000261501812	Color:1.6.108	Depth:1.6.79


### Set up devices
- each k4a device needs to be configured
- multi-camera rigs need to set up the WiredSyncMode
    - Master = WiredSyncMode.MASTER
    - Subordinate = WiredSyncMode.SUBORDINATE
    - subordinate_delay_off_master_usec=640 (master samples every 900usec)
- [Kinect SDK](https://microsoft.github.io/Azure-Kinect-Sensor-SDK/master/structk4a__device__configuration__t_a8208974f05d89fc1362c6a0900bdef4d.html#a8208974f05d89fc1362c6a0900bdef4d)

In [8]:
from pyk4a import (
    PyK4A,
    Config,
    ColorResolution,
    DepthMode,
    WiredSyncMode,
)

In [9]:
devices = {
    "master": {
        "id": 0,
        "pyk4a_config": {
            "color_resolution": ColorResolution.RES_720P,
            "depth_mode": DepthMode.NFOV_UNBINNED,
            #"synchronized_images_only": False,
            #"wired_sync_mode": WiredSyncMode.MASTER,
        },
        "process_kwargs": {
            "display_frames": True,
            "display_time": False,
            "save_color": True,
        },
    }
}

### Set up processing functions
- Functions for reducing video file sizes to 8 bit. 

In [10]:
_dtype = np.uint16

In [11]:
def process_depth(depth):
    #return np.clip((depth - 435) * (depth < 690), 0, 255).astype(_dtype)
    return depth.astype(_dtype)
def process_ir(ir):
    #ir = np.clip(ir + 100, 160, 5500)
    #return ((np.log(ir) - 5) * 70).astype(_dtype)
    return ir.astype(_dtype)

### Recording parameters

In [12]:
ir_depth_write_frames_kwargs={
        "codec": "ffv1", #"ffv1",
        "crf": 14,
        "threads": 6,
        "fps": 30,
        "slices": 24,
        "slicecrc": 1,
        "frame_size": None,
        "get_cmd": False,
    }

In [13]:
color_write_frames_kwargs={
        "codec": "ffv1", #"h264",
        "crf": 22,
        "threads": 6,
        "fps": 30,
        "slices": 24,
        "slicecrc": 1,
        "frame_size": None,
        "get_cmd": False,
    }

In [14]:
recording_duration = 60

### Run recording

In [15]:
start_recording(
    filename_prefix,
    recording_duration,
    devices=devices,
    depth_function = process_depth,
    ir_function = process_ir,
    ir_depth_dtype = _dtype,
    ir_depth_write_frames_kwargs=ir_depth_write_frames_kwargs,
    color_write_frames_kwargs=color_write_frames_kwargs
)

Recirding (s):   0%|          | 0/60 [00:00<?, ?it/s]

capture_from_azure initialized: master 
0 frame written 19.66350030899048
100 frame written 26.398544311523438
200 frame written 31.15181803703308
300 frame written 37.24355387687683
400 frame written 42.878641843795776
500 frame written 48.99902558326721
600 frame written 56.04684400558472
Framerate (master):29.8938
700 frame written 62.81228828430176
800 frame written 70.36214852333069
900 frame written 77.43817925453186
1000 frame written 85.03448963165283
1100 frame written 91.63912558555603
1200 frame written 98.11063313484192
1300 frame written 103.4655089378357
1400 frame written 111.58061242103577
1500 frame written 118.61526989936829


Process Process-1:
Traceback (most recent call last):
  File "/home/tim/anaconda3/envs/kinect_acquisition/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tim/anaconda3/envs/kinect_acquisition/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/n/groups/datta/tim_sainburg/projects/kinectacq/kinectacq/acquisition.py", line 102, in capture_from_azure
    capture = k4a.get_capture()
  File "/home/tim/anaconda3/envs/kinect_acquisition/lib/python3.8/site-packages/pyk4a/pyk4a.py", line 126, in get_capture
    _verify_error(res)
  File "/home/tim/anaconda3/envs/kinect_acquisition/lib/python3.8/site-packages/pyk4a/errors.py", line 18, in _verify_error
    raise K4AException()
pyk4a.errors.K4AException


### Read and show file

In [16]:
raise ValueError('Wait until start_recording finishes')

ValueError: Wait until start_recording finishes

In [ ]:
print('\n'.join([i.as_posix() for i in list((filename_prefix/'master').iterdir())]))

In [ ]:
system_timestamps = np.load(filename_prefix / 'master' / 'system_timestamps.npy')

In [ ]:
len(system_timestamps)

In [ ]:
len(system_timestamps) /((np.max(system_timestamps) - np.min(system_timestamps))*1e-9)

In [ ]:
((np.max(system_timestamps) - np.min(system_timestamps))*1e-9)

In [ ]:
np.load(filename_prefix / 'master' / 'depth_timestamps.npy')

In [ ]:
file_loc = filename_prefix / 'master' /  "depth.avi"
vidcap = cv2.VideoCapture(file_loc.as_posix())
success,image = vidcap.read()
total_frames = vidcap.get(7)
print(total_frames, image.shape)

In [ ]:
nbit = 16
total_expected_bits = total_frames * np.product(image.shape) * nbit
total_expected_MB_uncompressed = total_expected_bits * 1.192e-7 
print('uncompressed size: {} MB'.format(round(total_expected_MB_uncompressed)))
print('actual_size: {} MB'.format(round(os.path.getsize(file_loc) * 1e-6)))

In [ ]:
# get frame at number
vidcap.set(1, 140)
success,image = vidcap.read()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.matshow(np.log(1+image)[:,:,0])

In [ ]:
file_loc = filename_prefix / 'master' /  "color.avi"
vidcap = cv2.VideoCapture(file_loc.as_posix())
success,image = vidcap.read()
total_frames = vidcap.get(7)
print(total_frames, image.shape)

In [ ]:
nbit = 8
total_expected_bits = total_frames * np.product(image.shape) * nbit
total_expected_MB_uncompressed = total_expected_bits * 1.192e-7 
print('uncompressed size: {} MB'.format(total_expected_MB_uncompressed))
print('actual_size: {} MB'.format(os.path.getsize(file_loc) * 1e-6))

In [ ]:
success,image = vidcap.read()
# get frame at number
vidcap.set(1, 50)
success,image = vidcap.read()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(image)